In [9]:

!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai gradio pyannote.audio

import os
import re
from collections import defaultdict
from openai import OpenAI
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from pyannote.audio import Pipeline
import torch
import gradio as gr










In [18]:
whisper_asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    return_timestamps=True
)
print("Whisper pipeline:", whisper_asr)

diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")
print("Diarization pipeline:", diarization_pipeline)


Device set to use cuda:0


Whisper pipeline: <transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline object at 0x7db6bb908e50>


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.


/usr/local/lib/python3.11/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Diarization pipeline: <pyannote.audio.pipelines.speaker_diarization.SpeakerDiarization object at 0x7db6eaf070d0>


/usr/local/lib/python3.11/dist-packages/speechbrain/processing/features.py:1311: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stats = torch.load(path, map_location=device)


In [10]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)
openai_api_key = userdata.get('OPEN_API_KEY')
openai = OpenAI(api_key=openai_api_key)


In [11]:

AUDIO_MODEL = 'whisper-1'
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [12]:
whisper_asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    return_timestamps=True )

Device set to use cuda:0


In [13]:
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.


...

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [14]:

system_message = """
You are an Advanced AI assistant that produces detailed meeting minutes from transcripts.
Your task is to analyze the meeting transcript and extract key insights, including:
1. **Speaker Identification**: Identify different speakers and their contributions.
2. **Sentiment Analysis**: Detect the sentiment and emotion of the discussion.
3. **Action Items**: Extract tasks, decisions, and assignments.
4. **Topic Segmentation**: Breakdown the conversation into structured sections.
5. **Meeting Summary**: Generate a concise summary with key points.
Your output should be in **well-structured Markdown format**.
"""


In [15]:
def identify_speakers(audio_path):
    diarization = diarization_pipeline(audio_path)
    speaker_segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        speaker_segments.append({
            "speaker": speaker,
            "start": turn.start,
            "end": turn.end
        })
    return speaker_segments

# sentiments and emotions
def analyze_sentiments_and_emotions(text):
    sentiment_analyzer = pipeline('sentiment-analysis')
    emotion_analyzer = pipeline('text-classification', model='j-hartmann/emotion-english-distilroberta-base')
    sentiment = sentiment_analyzer(text[:512])[0]['label']
    emotion = emotion_analyzer(text[:512])[0]['label']
    return sentiment, emotion

# analyze speaker sentiments
def analyze_speaker_sentiments(speaker_texts):
    speaker_sentiments = {}
    for speaker, dialogues in speaker_texts.items():
        full_text = " ".join(dialogues)
        sentiment, emotion = analyze_sentiments_and_emotions(full_text)
        speaker_sentiments[speaker] = {"Sentiment": sentiment, "Emotion": emotion}
    return speaker_sentiments

# extract action items
def extract_action_items(transcript):
    action_keywords = ["approved", "scheduled", "decided", "assigned"]
    action_patterns = [
        r"(?P<action>approve|schedule|decide|assign) (?P<task>.+) by (?P<deadline>.+)",
        r"(?P<person>.+) will (?P<action>.+) by (?P<deadline>.+)"
    ]
    action_items = []
    for sentence in re.split(r'(?<=\.)\s+', transcript):
        for pattern in action_patterns:
            match = re.search(pattern, sentence, re.IGNORECASE)
            if match:
                action_items.append(match.groupdict())
    return action_items

# topics
def segment_topics(transcript):
    topic_keywords = ["whereas", "section", "motion", "resolution", "announcement", "agenda", "discussion", "conclusion", "approved", "decision", "summary", "update", "introduction"]
    topics = defaultdict(list)
    current_topic = "General Discussion"
    sentences = re.split(r'(?<=\.)\s+', transcript)
    for sentence in sentences:
        words = sentence.lower().split()
        for keyword in topic_keywords:
            if keyword in words:
                current_topic = sentence.strip()
                break
        topics[current_topic].append(sentence.strip())
    return topics


In [16]:
# Function to process audio and generate meeting minutes
def process_audio(audio_path):
    try:
        # Step 1: Transcription using Whisper
        transcription_result = whisper_asr(audio_path)
        transcription = " ".join([chunk['text'] for chunk in transcription_result['chunks']])  # Extract text only

        # Step 2: Speaker Identification using Diarization
        speaker_segments = identify_speakers(audio_path)
        speaker_texts = defaultdict(list)
        for segment in speaker_segments:
            speaker_texts[segment["speaker"]].append(transcription[int(segment["start"]):int(segment["end"])])

        # Step 3: Sentiment & Emotion Analysis
        speaker_sentiments = analyze_speaker_sentiments(speaker_texts)

        # Step 4: Extract Action Items
        action_items = extract_action_items(transcription)

        # Step 5: Topic Segmentation
        segmented_topics = segment_topics(transcription)

        # Step 6: Generate Meeting Summary
        user_prompt = f"""
        Below is a meeting transcript. Please generate **well-structured meeting minutes** in Markdown format.
        ### **Meeting Minutes**
        #### **Summary**
        - Briefly summarize the meeting objectives, discussions, and conclusions.
        ### **Attendees and Speakers**
        - List key participants.
        ### **Sentiments**
        - Get the emotional tone of the speaker.
        ### **Key Takeaways**
        - Important insights gained from the meeting.
        ### **Action Items & Decisions**
        - Mention the responsibilities of each person.
        - Include deadlines if mentioned.
        **Transcript:**
        {transcription}
        """

        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "system", "content": system_message},
                      {"role": "user", "content": user_prompt}]
        )

        # Formatting the Output
        formatted_output = f"## **Meeting Summary**\n{response.choices[0].message.content}\n\n"
        formatted_output += "## **Speaker Identification & Sentiments**\n"
        for speaker, analysis in speaker_sentiments.items():
            formatted_output += f"🎙️ **{speaker}** → Sentiment: {analysis['Sentiment']} | Emotion: {analysis['Emotion']}\n"
        formatted_output += "\n## **Action Items & Decisions**\n"
        for i, item in enumerate(action_items, 1):
            formatted_output += f"{i}. **{item['action'].title()}**: {item['task']} (Deadline: {item['deadline']})\n"
        formatted_output += "\n## **Segmented Topics**\n"
        for topic, discussions in segmented_topics.items():
            formatted_output += f"### 🔹 {topic}\n"
            for discussion in discussions:
                formatted_output += f"   - {discussion}\n"

        return transcription, formatted_output

    except Exception as e:
        return f"An error occurred: {str(e)}", ""

In [17]:

# Create Gradio Interface
gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),
    outputs=[gr.Textbox(label="Transcribed Text"), gr.Markdown(label="Generated Meeting Minutes")],
    title="🎤 Meeting Minutes Generator",
    description="Upload an audio file and get structured meeting minutes automatically.",
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d037211d9a95da21af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
